# Toxic Comment Classification with BERT

This notebook implements a multi-label toxic comment classifier using BERT, PyTorch, and Hugging Face Transformers.

## 1. Install Dependencies
Run this cell only once to install required packages.

In [ ]:
!pip install transformers
!pip install torch torchvision torchaudio
!pip install pandas scikit-learn

## 2. Import Libraries and Setup

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import re
from tqdm import tqdm

# Reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

## 3. Load Dataset

In [ ]:
df = pd.read_csv('train.csv', engine='python')
print(df.head())
print(df.shape)
print(df.isnull().sum())

## 4. Clean Text Data

In [ ]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # remove URLs
    text = re.sub(r'\n', ' ', text)      # remove newlines
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # remove special chars
    return text.lower()

df['comment_text'] = df['comment_text'].astype(str).apply(clean_text)
print(df['comment_text'].head())

## 5. Define Labels and Convert to Numeric

In [ ]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

for col in labels:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

## 6. Train-Validation Split

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)

## 7. Load BERT Tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Test tokenization
sample = train_df['comment_text'].iloc[0]
tokens = tokenizer.encode_plus(
    sample,
    max_length=128,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='pt'
)
print(tokens['input_ids'])

## 8. Dataset Class for PyTorch DataLoader

In [ ]:
class ToxicCommentsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, labels):
        self.df = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.labels = labels

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.df.iloc[index]['comment_text']
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )
        targets = torch.tensor(self.df.iloc[index][self.labels].values, dtype=torch.float)
        return {
            'input_ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'token_type_ids': torch.tensor(inputs['token_type_ids'], dtype=torch.long),
            'targets': targets
        }

## 9. Initialize DataLoaders

In [ ]:
MAX_LEN = 128
BATCH_SIZE = 16

train_dataset = ToxicCommentsDataset(train_df.reset_index(drop=True), tokenizer, MAX_LEN, labels)
val_dataset = ToxicCommentsDataset(val_df.reset_index(drop=True), tokenizer, MAX_LEN, labels)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Quick check
data = next(iter(train_loader))
print(data['input_ids'].shape)  # [BATCH_SIZE, MAX_LEN]
print(data['targets'].shape)    # [BATCH_SIZE, 6]